In [4]:
#imports
import os
import gc
import pandas as pd
import numpy as np
import datetime as dt 

import matplotlib.pyplot as plt
import matplotlib.cm as cm 
import seaborn as sns

import plotly.graph_objects as go 
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.offline 

from colorama import Fore, Style, init 
from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

In [5]:
def PrintColor(text: str, color=Fore.BLUE, style=Style.BRIGHT):
    "Prints color outputs using colorama using a text F-string"
    print(style + color + text + Style.RESET_ALL)

def summarize_dataframe(df):
    summary_df = pd.DataFrame(df.dtypes, columns=['dtypes'])
    summary_df['missing#'] = df.isna().sum().values*100
    summary_df['missing%'] = (df.isna().sum().values*100)/len(df)
    summary_df['uniques'] = df.nunique().values
    summary_df['first_value'] = df.iloc[0].values
    summary_df['last_value'] = df.iloc[len(df)-1].values
    summary_df['count'] = df.count().values

    desc = pd.DataFrame(df.describe().T)
    summary_df['min'] = desc['min']
    summary_df['max'] = desc['max']
    summary_df['mean'] = desc['mean']
    return summary_df